In [2]:
#imports
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import re
import time
import random
import sys
pd.set_option('display.max_colwidth', None)

from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
from scipy import sparse
from matplotlib import pyplot as plt
import utils as ut

In [42]:
def rec_search(category, query, wout='no'):

    if category.lower() == 'video games':
        lookup, recommender = pd.read_pickle('./pickles/videog_look.pkl'),pd.read_pickle('./pickles/videog_rec.pkl')
    elif category.lower() == 'movies':
        lookup, recommender = pd.read_pickle('./pickles/movies_look.pkl'), pd.read_pickle('./pickles/movie_rec.pkl') 
    elif category.lower() == 'books':
        lookup, recommender = pd.read_pickle('./pickles/books_look.pkl'), pd.read_pickle('./pickles/books_rec.pkl')
    else:
        return "Sorry, that wasn't one of the available categories"

    try:
        query=query.title() #uppercase first letter of each word in the query in case it's not entered that way
        titles = list(lookup[lookup['product_title'].str.contains(query)]['product_title'])
        q = titles[0] #this is the item to search for

        if wout.lower() == 'no':
            query_dict = dict(recommender.loc[q].sort_values())
            
            final_printout = ''
            for key in list(query_dict.keys())[1:11]:
                final_printout += '\n' #add new line
                final_printout += f'{key}' #print item name
                final_printout += '\n'
                final_printout += f"""
                This item has {round(lookup[lookup['product_title']==key]['tot_prod_reviews'].mean())} reviews 
                and a {round(lookup[lookup['product_title']==key]['avg_prod_stars'].mean(), 2)} average star rating"""


            return f'''

            Recommending items similar to: {q}
            {q} has {round(lookup[lookup['product_title']==q]['tot_prod_reviews'].mean())} reviews
            and a {round(lookup[lookup['product_title']==q]['avg_prod_stars'].mean(), 2)} average star rating.

            Here are the 10 recommended items for you based on your search parameters:
            
            {final_printout}
            
            '''
            
        else:
            
            wout = wout.title() #capitlize first letters
            query_dict = dict(recommender.loc[q].sort_values())
            filtered_query = [] #make empty list
            for key, value in query_dict.items(): #index into dictionary of results
                if wout not in key: #check if avoided keyword is in results
                    filtered_query.append((key, value)) #make list of results that DON'T include "wout" keyword
            
            final_printout = ''
            for item in filtered_query[1:11]:
                final_printout += '\n' #add new line
                final_printout += f'{item[0]}' #print item name
                final_printout += '\n'
                final_printout += f"""
                This item has {round(lookup[lookup['product_title']==item[0]]['tot_prod_reviews'].mean())} reviews 
                and a {round(lookup[lookup['product_title']==item[0]]['avg_prod_stars'].mean(), 2)} average star rating"""
            
            return f'''

            Recommending items similar to: {q}
            {q} has {round(lookup[lookup['product_title']==q]['tot_prod_reviews'].mean())} reviews
            and a {round(lookup[lookup['product_title']==q]['avg_prod_stars'].mean(), 2)} average star rating.

            Here are the 10 recommended items for you based on your search EXCLUDING "{wout}":
            
            {final_printout}
            
            '''
        
    except:
        return f'Sorry, "{query}" does not appear to be in the product database'

In [43]:
rec_search('books', 'batman', 'batman')

'\n\n            Recommending items similar to: Batman: The Dark Knight Returns\n            Batman: The Dark Knight Returns has 177 reviews\n            and a 4.64 average star rating.\n\n            Here are the 10 recommended items for you based on your search EXCLUDING "Batman":\n            \n            \nV for Vendetta\n\n                This item has 92 reviews \n                and a 4.54 average star rating\nDaredevil Visionaries - Frank Miller, Vol. 1\n\n                This item has 18 reviews \n                and a 4.28 average star rating\nMarvels\n\n                This item has 31 reviews \n                and a 4.48 average star rating\nFrank Miller\'s Ronin\n\n                This item has 24 reviews \n                and a 4.38 average star rating\nInfinity Gauntlet TPB\n\n                This item has 12 reviews \n                and a 4.58 average star rating\nNew X-Men Vol. 1: E is for Extinction (v. 1)\n\n                This item has 27 reviews \n              